/*
 * Gibbs sample for Sequence feature
 * Copyright (c) 2018-2019 Wei Zhou, ChaoRui Liu, All Rights Reserved.
 *
 * Permission is hereby granted, free of charge, to any person
 * obtaining a copy of this software and associated documentation
 * files (the "Software"), to deal in the Software without
 * restriction, including without limitation the rights to use,
 * copy, modify, merge, publish, distribute, sublicense, and/or sell
 * copies of the Software, and to permit persons to whom the
 * Software is furnished to do so, subject to the following
 * conditions:
 *
 * The above copyright notice and this permission notice shall be
 * included in all copies or substantial portions of the Software.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
 * EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
 * OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
 * NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 * HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
 * WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
 * FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
 * OTHER DEALINGS IN THE SOFTWARE.
 */

In [55]:
from numpy.random import randint
from numpy.random import rand
import numpy as np

In [56]:
def calc_score(seq,motif_length,p,q):
    N = len(seq)
    qx = [1]*(N-motif_length+1)
    px = [1]*(N-motif_length+1)
    for j in xrange(N-motif_length+1):
        for k in xrange(motif_length):
            c = seq[j+k]
            qx[j] = qx[j] * q[c][k]
            px[j] = px[j] * p[c]
            
    Aj = [x/y for (x,y) in zip(qx, px)]
    norm_c = sum(Aj)
    return map(lambda x: x/norm_c, Aj)

In [57]:
def gibbs(DNA_seq, w):
    
    K = len(DNA_seq)
    
    amino_acid = []
    for seq in DNA_seq:
        for acid in seq:
            amino_acid.append(acid)
    
    alp = list(set(amino_acid))
    
    pos = [randint(0, len(seq)-w+1) for seq in DNA_seq]  

    end_position = None
    while end_position != pos:
        end_position = pos[:]
        for i in xrange(K):

            seq_minus = DNA_seq[:]
            del seq_minus[i]
            pos_minus = pos[:]
            del pos_minus[i]
            q, p = compute_model(seq_minus, pos_minus, alp, w)


            Aj = calc_score(DNA_seq[i], w,p,q)
            pos[i] = sample(range(len(DNA_seq[i])-w+1), Aj)
    return pos

In [58]:
def compute(DNA_seq, pos, alp, w):

    q = {x:[1]*w for x in alp}
    p = {x: 1 for x in alp}
    
    for i in xrange(len(DNA_seq)):
        start_pos = pos[i]        
        for j in xrange(w):
            c = DNA_seq[i][start_pos+j]
            q[c][j] += 1
    for c in alp:
        for j in xrange(w):
            q[c][j] = q[c][j] / float( len(DNA_seq)+len(alp) )
    for i in xrange(len(DNA_seq)):
        for j in xrange(len(DNA_seq[i])):
            if j < pos[i] or j > pos[i]+w:
                c = DNA_seq[i][j]
                p[c] += 1
    total = sum(p.values())
    for c in alp:
        p[c] = p[c] / float(total)
    
    return q, p

In [59]:
def sample(alp, dist):
    sample = None
    cum_dist = np.cumsum(dist)
    r = rand()
    for i in xrange(len(dist)):
        if r < cum_dist[i]:
            sample = alp[i]
            break
    
    return sample

In [60]:
seqs = ['GAGTATACCT']
k = 3
print gibbs(seqs, 7)

[2]
